<a href="https://colab.research.google.com/github/AhwuIOT/ai_proj/blob/main/using_faiss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import fitz  # PyMuPDF

model = SentenceTransformer('all-MiniLM-L6-v2')
texts = []
sources = []
index = None

def extract_chunks_from_pdf(pdf_path, chunk_size=300):
    doc = fitz.open(pdf_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text()
    doc.close()
    return [full_text[i:i+chunk_size].strip() for i in range(0, len(full_text), chunk_size) if full_text[i:i+chunk_size].strip()]

def add_pdf_to_faiss(pdf_path):
    global index, texts, sources
    chunks = extract_chunks_from_pdf(pdf_path)
    embeddings = model.encode(chunks).astype('float32')
    if index is None:
        index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    texts.extend(chunks)
    sources.extend([pdf_path] * len(chunks))
    return len(chunks)

def search(query, k=3):
    if index is None or index.ntotal == 0:
        return []
    query_vec = model.encode([query]).astype('float32')
    D, I = index.search(query_vec, k)
    return [{"source": sources[i], "content": texts[i]} for i in I[0]]


In [ ]:
add_pdf_to_faiss("/content/genc2021-dac.pdf")  # 替換為你上傳的檔案名
results = search("Gemmini")
for r in results:
    print("📄", r["source"])
    print("📝", r["content"])
    print("-" * 50)


📄 /content/genc2021-dac.pdf
📝 requirements, Gemmini provides a multi-level programming interface
to satisfy users with different requirements. In addition, Gemmini
is the ﬁrst infrastructure that provides hardware support for virtual
memory without the need for any special driver software, making it
signiﬁcantly easier for end-
--------------------------------------------------
📄 /content/genc2021-dac.pdf
📝 requirements, Gemmini provides a multi-level programming interface
to satisfy users with different requirements. In addition, Gemmini
is the ﬁrst infrastructure that provides hardware support for virtual
memory without the need for any special driver software, making it
signiﬁcantly easier for end-
--------------------------------------------------
📄 /content/genc2021-dac.pdf
📝 te
a complete SoC environment that can boot Linux, directly enabling
architects to evaluate subtle trade-offs at the system level.
III. GEMMINI GENERATOR
Gemmini is an open-source, full-stack generator of DNN